<a href="https://colab.research.google.com/github/liangli217/LLM_learning/blob/main/Langchain_LLM_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install python-dotenv
!pip install openai

In [34]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

In [35]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [36]:
def get_completion(prompt, model=llm_model):
    client = openai.OpenAI() # Initialize the client with API key
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create( # Use the new API call
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content # Access content correctly

In [37]:
from google.colab import userdata
# Set the OpenAI API key from Colab secrets
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [38]:
get_completion("What is 1+1?")

'1+1 equals 2.'

In [39]:
!pip install langchain_openai
from langchain_openai import ChatOpenAI

In [40]:
from langchain_openai import ChatOpenAI

In [41]:
chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7ee3fbd1da30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7ee4005c3080>, root_client=<openai.OpenAI object at 0x7ee408494fb0>, root_async_client=<openai.AsyncOpenAI object at 0x7ee3fbd1c770>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [42]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [43]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [44]:
## Chains

In [45]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [46]:
#!pip install pandas

In [47]:
import pandas as pd
df = pd.read_csv('sample_data/Data.csv')

In [48]:

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [49]:
llm = ChatOpenAI(temperature=0.9)


In [50]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
a company that makes {product}"
)

In [51]:
chain = LLMChain(llm=llm, prompt=prompt)

In [52]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Royal Linens Co."'

**Simple Sequential Chain**

In [54]:
from langchain.chains import SimpleSequentialChain

In [55]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [56]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [57]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [58]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royal Comfort Linens"
"Royal Comfort Linens offers luxurious bedding and bath linens that provide the perfect blend of style, comfort, and elegance."

> Finished chain.


'"Royal Comfort Linens offers luxurious bedding and bath linens that provide the perfect blend of style, comfort, and elegance."'

**Sequential** **Chain**

In [59]:
from langchain.chains import SequentialChain

In [64]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
fist_prompt = ChatPromptTemplate.from_template("Translate the following review to english: \n\n{Review}")

chain_one = LLMChain(llm=llm, prompt=fist_prompt, output_key="English_Review")


In [61]:
second_prompt=ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence: \n\n{English_Review}")
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [62]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [63]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [67]:
# overall_chain

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables= ['Review'],
    output_variables=['English_Review', 'summary', 'language','followup_message'],
    verbose=True


)

In [68]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


In [70]:
overall_chain(df.Review[5])

/tmp/ipython-input-3243744767.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(df.Review[5])




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product mediocre, the foam does not hold well, and suggests that it may be an old batch or counterfeit.',
 'language': 'French',
 'followup_message': "Je vous remercie pour votre avis honnête sur le produit. Je suis désolé d'apprendre que vous avez trouvé le goût médiocre et que la mousse ne tient pas bien. Il est possible qu'il s'agisse d'un lot ancien ou contrefait. Nous allons enquêter sur ce problème et nous assurer que nos clients reçoivent des produits de qualité. Merci de nous avoir informés et nous espérons vous offrir une meilleure expérience à l'avenir."}

**Router Chain**

In [71]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [72]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [73]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain
from langchain.prompts import PromptTemplate